# Trenowanie RL do balansowania na Cartpole

Ten notatnik jest częścią [Kursu AI dla Początkujących](http://aka.ms/ai-beginners). Został zainspirowany przez [oficjalny samouczek PyTorch](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) oraz [tę implementację Cartpole w PyTorch](https://github.com/yc930401/Actor-Critic-pytorch).

W tym przykładzie użyjemy RL, aby wytrenować model do balansowania drążka na wózku, który może poruszać się w lewo i w prawo po poziomej skali. Wykorzystamy środowisko [OpenAI Gym](https://www.gymlibrary.ml/) do symulacji drążka.

> **Note**: Możesz uruchomić kod z tej lekcji lokalnie (np. w Visual Studio Code), w takim przypadku symulacja otworzy się w nowym oknie. Jeśli uruchamiasz kod online, może być konieczne wprowadzenie pewnych zmian w kodzie, jak opisano [tutaj](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7).

Zaczniemy od upewnienia się, że Gym jest zainstalowany:


In [ ]:
import sys
!{sys.executable} -m pip install gym

Teraz stwórzmy środowisko CartPole i zobaczmy, jak na nim operować. Środowisko ma następujące właściwości:

* **Action space** to zbiór możliwych akcji, które możemy wykonać na każdym kroku symulacji  
* **Observation space** to przestrzeń obserwacji, które możemy dokonać  


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Zobaczmy, jak działa symulacja. Poniższa pętla uruchamia symulację, dopóki `env.step` nie zwróci flagi zakończenia `done`. Będziemy losowo wybierać akcje za pomocą `env.action_space.sample()`, co oznacza, że eksperyment prawdopodobnie zakończy się bardzo szybko niepowodzeniem (środowisko CartPole kończy się, gdy prędkość CartPole, jego pozycja lub kąt wykraczają poza określone limity).

> Symulacja otworzy się w nowym oknie. Możesz uruchomić kod kilka razy i zobaczyć, jak się zachowuje.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

Możesz zauważyć, że obserwacje zawierają 4 liczby. Są to:
- Pozycja wózka
- Prędkość wózka
- Kąt drążka
- Szybkość obrotu drążka

`rew` to nagroda, którą otrzymujemy na każdym kroku. W środowisku CartPole otrzymujesz 1 punkt za każdy krok symulacji, a celem jest maksymalizacja całkowitej nagrody, czyli czasu, przez który CartPole potrafi utrzymać równowagę bez przewrócenia się.

Podczas uczenia ze wzmocnieniem naszym celem jest wytrenowanie **polityki** $\pi$, która dla każdego stanu $s$ powie nam, jaką akcję $a$ należy podjąć, czyli zasadniczo $a = \pi(s)$.

Jeśli chcesz podejścia probabilistycznego, możesz traktować politykę jako zwracającą zestaw prawdopodobieństw dla każdej akcji, czyli $\pi(a|s)$ oznaczałoby prawdopodobieństwo, że powinniśmy podjąć akcję $a$ w stanie $s$.

## Metoda Gradientu Polityki

W najprostszym algorytmie RL, zwanym **Gradientem Polityki**, będziemy trenować sieć neuronową do przewidywania kolejnej akcji.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

Będziemy trenować sieć, przeprowadzając wiele eksperymentów i aktualizując naszą sieć po każdym uruchomieniu. Zdefiniujmy funkcję, która przeprowadzi eksperyment i zwróci wyniki (tzw. **ślad**) - wszystkie stany, akcje (i ich zalecane prawdopodobieństwa) oraz nagrody:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

Możesz uruchomić jeden epizod z nieprzeszkoloną siecią i zaobserwować, że całkowita nagroda (czyli długość epizodu) jest bardzo niska:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

Jednym z trudnych aspektów algorytmu gradientu polityki jest użycie **zdyskontowanych nagród**. Pomysł polega na obliczeniu wektora całkowitych nagród na każdym kroku gry, a podczas tego procesu dyskontujemy wcześniejsze nagrody za pomocą współczynnika $gamma$. Normalizujemy również wynikowy wektor, ponieważ będziemy go używać jako wagi wpływającej na nasze szkolenie:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

Teraz przejdźmy do właściwego treningu! Przeprowadzimy 300 epizodów, a w każdym z nich wykonamy następujące kroki:

1. Uruchom eksperyment i zbierz ślad.
2. Oblicz różnicę (`gradients`) między podjętymi działaniami a przewidywanymi prawdopodobieństwami. Im mniejsza różnica, tym bardziej jesteśmy pewni, że podjęliśmy właściwe działanie.
3. Oblicz zdyskontowane nagrody i pomnóż gradienty przez zdyskontowane nagrody - to zapewni, że kroki z wyższymi nagrodami będą miały większy wpływ na ostateczny wynik niż te z niższymi nagrodami.
4. Oczekiwane docelowe działania dla naszej sieci neuronowej będą częściowo pochodzić z przewidywanych prawdopodobieństw podczas eksperymentu, a częściowo z obliczonych gradientów. Użyjemy parametru `alpha`, aby określić, w jakim stopniu gradienty i nagrody są brane pod uwagę - nazywa się to *współczynnikiem uczenia* algorytmu wzmacniającego.
5. Na koniec trenujemy naszą sieć na stanach i oczekiwanych działaniach, a następnie powtarzamy proces.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

Teraz uruchommy odcinek z renderowaniem, aby zobaczyć wynik:


In [ ]:
_ = run_episode(render=True)

Mam nadzieję, że teraz widać, że drążek potrafi całkiem dobrze utrzymać równowagę!

## Model Aktor-Krytyk

Model Aktor-Krytyk to rozwinięcie metod gradientów polityki, w którym budujemy sieć neuronową, aby nauczyć się zarówno polityki, jak i szacowanych nagród. Sieć będzie miała dwa wyjścia (lub można ją traktować jako dwie oddzielne sieci):
* **Aktor** będzie rekomendował akcję do podjęcia, dostarczając nam rozkład prawdopodobieństwa stanów, podobnie jak w modelu gradientów polityki.
* **Krytyk** oszacuje, jakie nagrody można uzyskać z tych akcji. Zwraca całkowite szacowane nagrody w przyszłości dla danego stanu.

Zdefiniujmy taki model:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

Musielibyśmy nieco zmodyfikować nasze funkcje `discounted_rewards` i `run_episode`:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


Teraz uruchomimy główną pętlę treningową. Użyjemy ręcznego procesu treningu sieci, obliczając odpowiednie funkcje strat i aktualizując parametry sieci:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## Kluczowe informacje

W tym demo zapoznaliśmy się z dwoma algorytmami uczenia ze wzmocnieniem: prostym gradientem polityki oraz bardziej zaawansowanym aktor-krytyk. Możesz zauważyć, że te algorytmy operują na abstrakcyjnych pojęciach stanu, akcji i nagrody – dzięki temu mogą być stosowane w bardzo różnych środowiskach.

Uczenie ze wzmocnieniem pozwala nam znaleźć najlepszą strategię rozwiązania problemu, analizując jedynie końcową nagrodę. Fakt, że nie potrzebujemy oznakowanych zbiorów danych, umożliwia wielokrotne powtarzanie symulacji w celu optymalizacji naszych modeli. Niemniej jednak, wciąż istnieje wiele wyzwań w obszarze RL, o których możesz się dowiedzieć, jeśli zdecydujesz się bardziej zgłębić tę fascynującą dziedzinę sztucznej inteligencji.



---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby tłumaczenie było precyzyjne, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za wiarygodne źródło. W przypadku informacji o kluczowym znaczeniu zaleca się skorzystanie z profesjonalnego tłumaczenia przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
